# Introduction

<figure style="display: table; text-align:center; margin-left: auto; margin-right:auto">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/simphony/docs/v4.0.0?filepath=docs%2Fusage%2Fsessions%2Fintroduction.ipynb "Click to run this tutorial yourself!")
    
</figure>

In SimPhoNy, [assertional knowledge](../assertional_knowledge.ipynb) is stored in _sessions_. You may think of a session as a "box" were [ontology individuals](../assertional_knowledge.ipynb#Ontology-individual-objects) can be placed. But sessions go beyond just storing assertional knowledge. Sessions can be connected to [SimPhoNy Wrappers](../wrappers/index.md). Each wrapper is a piece of software that seamlessly translates the assertional knowledge to a form that is compatible with a specific simulation engine, database, data repository or file format.

In order to keep things simple, this section focuses on sessions that are not connected to any wrapper. All the information stored in such sessions is stored in the computer's volatile memory and lost when the Python shell is closed. After having read this section, you can head to the [next one](../wrappers/index.md) to learn more about SimPhoNy Wrappers.

All ontology individuals in SimPhoNy are stored in a session. The session where an individual is stored is always accesible through the [session](../../api_reference.md#simphony_osp.ontology.OntologyEntity.session) attribute, which is **writable**. In fact, changing this attribute is one of the several ways to transfer an ontology individual between sessions.

In [1]:
from simphony_osp.namespaces import city

freiburg = city.City(name="Freiburg", coordinates=[47.997791, 7.842609])
freiburg.session

But where are newly created individuals stored? Indeed, when a new individual is created, it has to be stored in a session. It is possible to pass a [session object](../../api_reference.md#simphony_osp.session.Session) to the call using the `session` keyword argument to choose where the individual will be stored.

In [2]:
from simphony_osp.session import Session

session_A = Session()  # create a new session
paris = city.City(
    name="Paris", coordinates=[48.85333, 2.34885],
    session=session_A
)
session_A, paris in session_A, freiburg in session_A

(<simphony_osp.session.session.Session at 0x7f0bee95ee80>, True, False)

But such argument is optional. When it is not specified, the individual is stored on the so-called _default session_. Every time you work with SimPhoNy in a Python shell, it creates a new session, called _Core Session_ and sets it as the default session. The default session can be changed later to any of your choice. The [Core Session](../../api_reference.md#simphony_osp.session.core_session) can be accessed at any time by importing it from the `simphony_osp.session` module.

In [3]:
from simphony_osp.session import core_session

core_session, freiburg in core_session, paris in core_session

(<simphony_osp.session.session.Session at 0x7f0c2c11f940>, True, False)

<div class="admonition note">
<div class="admonition-title" style="font-weight: bold"><div style="display: inline-block">Note</div></div>

The interface of the [session object](../api_reference.md#simphony_osp.session.Session) has been designed to interact exclusively with ontology individuals, and therefore, this page only shows you how to use sessions to deal with assertional knowledge.
    
However, there is no technical reason for such limitation. Sessions can actually store any ontology entity (including terminological knowledge). As a curiosity, the entities from the ontologies that you install using [pico](ontologies/pico.md) are stored in a hidden session that is not meant to be directly accessed.

</div>

The default session can be temporarily changed using the `with` statement.

In [4]:
session_B = Session()

# this will be explained later
session_A.locked = True
session_B.locked = True

with session_B:
    london = city.City(name="London", coordinates=[51.50737, -0.12767])
    
print(paris in session_A, london in session_B)

# Be careful when using the with statement with several session objects:
# keep in mind that the second will be the one set as default.
with session_A, session_B:
    default_session = Session.get_default_session()
    print(default_session is session_A, default_session is session_B)

True True
False True


Sessions actually work in a way similar to databases. To start using them, one first has to "open" or "connect" to them. After that, changes can be performed on the data they contain, but such changes are not made permanent until a "commit" is performed. When one finishes working with them, the connection should be "closed". Unconfirmed changes are lost when the connection is "closed".

In SimPhoNy, all sessions are automatically "opened" when they are created. The "commit" and "close" operations are controlled manually.

In spite of that general rule, for sessions that are not connected to a wrapper, which are the ones being illustrated in this page, the "commit" command actually does nothing, as confirmed changes have nowhere else to go and be made permanent. You can think of commits being automatic in this case. These sessions also do not implement the "close" command.

Therefore, this general rule has just been introduced in order to present a useful mental model for working with _all_ sessions, which includes [sessions connected to a wrapper](../wrappers/index.md).

Having said that, it is now simpler to understand the purpose of the [locked attribute of session objects](../../api_reference.md#simphony_osp.session.Session.locked) that appears in the last example. The `with` statement not only sets a session as the default, but also _closes_ it when leaving its context. _Locking a session_ with the [locked](../../api_reference.md#simphony_osp.session.Session.locked) attribute prevents the session from being closed if one intents to continue using it. To restore the original behavior, set it to `False`.